# RNA Structure Generation

Some random structures generated using the defined Context-Free Grammar (CFG).

In [3]:
import random
import sys


sys.setrecursionlimit(10000)

def countS(n,cache,weight_unpaired,weight_stack):
    if ("S",n) not in cache:
        if n == 0:
            val = 1
        else:
            val = weight_unpaired*countS(n-1,cache,weight_unpaired,weight_stack)
            if n>1:
                val += countS(n-2,cache,weight_unpaired,weight_stack)
            if n>2:
                for i in range(1,n-1):
                    val += countT(i,cache,weight_unpaired,weight_stack)*countS(n-2-i,cache,weight_unpaired,weight_stack)
        cache[("S",n)] = val
    return cache[("S",n)]
    
def generateS(n,cache,weight_unpaired,weight_stack):
    if n == 0:
        return ""
    else:
        r = random.random() * countS(n,cache,weight_unpaired,weight_stack)
        r -= weight_unpaired * countS(n-1,cache,weight_unpaired,weight_stack)
        if r<0:
            return "." + generateS(n-1,cache,weight_unpaired,weight_stack)
        if n>1:
            r -= countS(n-2,cache,weight_unpaired,weight_stack)
            if r<0:
                return "()" + generateS(n-2,cache,weight_unpaired,weight_stack)
        if n>2:
            for i in range(1,n-1):
                r -= countT(i,cache,weight_unpaired,weight_stack)*countS(n-2-i,cache,weight_unpaired,weight_stack)
                if r<0:
                    return "("+generateT(i,cache,weight_unpaired,weight_stack)+")"+generateS(n-2-i,cache,weight_unpaired,weight_stack)
    return None

def countT(n,cache,weight_unpaired,weight_stack):
    if ("T",n) not in cache:
        if n == 0:
            val = 0
        else:
            val = weight_unpaired*countS(n-1,cache,weight_unpaired,weight_stack)
            if n==2:
                val += weight_stack
            if n>2:
                val += countT(n-2,cache,weight_unpaired,weight_stack)
            if n>2:
                val += weight_stack*countT(n-2,cache,weight_unpaired,weight_stack)
            if n>3:
                for i in range(1,n-2):
                    val += countT(i,cache,weight_unpaired,weight_stack)*countT(n-2-i,cache,weight_unpaired,weight_stack)
        cache[("T",n)] = val
    return cache[("T",n)]
    
def generateT(n,cache,weight_unpaired,weight_stack):
    if n == 0:
        return "#"
    else:
        r = random.random()*countT(n,cache,weight_unpaired,weight_stack)
        r -= weight_unpaired*countS(n-1,cache,weight_unpaired,weight_stack)
        if r<0:
            return "."+generateS(n-1,cache,weight_unpaired,weight_stack)
        if n==2:
            r -= weight_stack
            if r<0:
                return "()"
        if n>2:
            r -= countT(n-2,cache,weight_unpaired,weight_stack)
            if r<0:
                return "()"+generateT(n-2,cache,weight_unpaired,weight_stack)
        if n>2:
            r -= weight_stack*countT(n-2,cache,weight_unpaired,weight_stack)
            if r<0:
                return "("+generateT(n-2,cache,weight_unpaired,weight_stack)+")"
        if n>3:
            for i in range(1,n-2):
                r -= countT(i,cache,weight_unpaired,weight_stack)*countT(n-2-i,cache,weight_unpaired,weight_stack)
                if r<0:
                    return "("+generateT(i,cache,weight_unpaired,weight_stack)+")"+generateT(n-2-i,cache,weight_unpaired,weight_stack)
    return None

In [5]:
N_STRUCTURES = 20
LENGTH = 100
WEIGHT_UNPAIRED = 1.0
WEIGHT_STACK = 10

cache = {}
structures = []

print(f"Generating {N_STRUCTURES} structures of length {LENGTH}...")
for i in range(N_STRUCTURES):
    s = generateS(LENGTH, cache, WEIGHT_UNPAIRED, WEIGHT_STACK)
    structures.append(s)
    print(f"{i+1}: {s}")

with open("generated_structures.txt", "w") as f:
    for s in structures:
        f.write(s + "\n")

print("\nStructures saved to 'generated_structures.txt'")

Generating 20 structures of length 100...
1: ((((((((((())))))((()))((()((((...))))))(((((((()((((()))())(.)))(((((((.()(.))))))()))))))))))..)))
2: .(((()))(()(((((((((()(((((())((((...))))))))))))(...((((())((()(((()))(.)(()))).))(())))))))))))...
3: ()(((())((((((())(((((((((())(())())((((....))))))))))))())(((((((())))))))(()))((()))))))..(((())))
4: (((((((()()((()).)((()))))))))(((((((()...()))((((((()(((((((((.))))(((.))))))))(.)..))))))())))))).
5: .(((((((((((...))(((((((((()))())))))())))(((()(((((((((()(.(()))))))))))))(()())))))))(())))()))...
6: ((((()())((()(.)))(((()).)()())()))((((((((((((((.(()))))))))))))))((.))(..(((()((((()))))))))))(())
7: (((((((((((((((((())))(())))))))())((((((()).((()()))))))()(((((((((()).)))))))))))))))(((())()))())
8: .((((()(.))))((((.)(.((((((((((.)((((.(()))))))(())((...(()(((()))))))))))((())))))))))(((()).()))))
9: (((.(()))(.)(((()()((((())))(((()))))((())(()(.))))))))(..((((()(())))))(((((((...))))))(()(.))))(.)
10: (()((((((()(((((.)